# Schwachstellenanalyse hinsichtlich bekannter CVE's

## Fragestellung

Treten in dem vorliegendem Git-Projekt mögliche Sicherheitslücken aufgrund von veralteten/fehlerhaften Bibliotheken & Frameworks auf? Wie kritisch werden jene Schwachstellen eingeschätzt?

### Relevanz
* historische Systeme tendieren zu veralteten Versionen von Bibliotheken/Frameworks/Artefakten --> kritisch bzgl. Erhalt von qualitativen Softwareeigenschaften, Schutzzielen der Informationssicherheit (und beispielsweise auch Zertifizierung im Unternehmensumfeld)
* relevant für Kunden/Anwender und Provider/Entwickler zugleich --> Ausnutzung von Schwachstellen von Dritten um Angriffe wie Sniffer-Angriffe, Denial-of-Service oder auch Buffer-Overflows zu erzeugen
* Konsequenz kann z.B. Datenverluste, unautorisierter Datenzugriff, Vertrauensverlust sein
* regelmäßige Checks und Updates reduzieren Aufwände im Vergleich zu Updates, welche mehrere Versionen überspringen
* Verkaufsargument in mancher Branche

### mögliche Konsequenzen
Dienstleister:
* weiterführende Analyse und Risikobewertung möglich: Welche Artefakte sind für uns relevant? Welche müssen inwieweit priorisiert werden? Wann kann ein Update eingeplannt werden? Wann ist die letzte Deadline? Welche Handlungsaktionen müssen fehlenden Bugfixes getätigt werden?
* Erstellung einer Aufwandsschätzung bzgl. Entwicklung, Updatemigration, Test, Integration, Auslieferung & möglicher Inkludierung in das nächste Release oder Light-Release/Bugfix/Snapshot

Kunden:
* Möglichkeit Druck für Handlungsbedarf beim Dienstleister auszuüben
* Worst-Case: Möglichkeit über Anbieterwechsel nachzudenken

## Datenquelle
* Java-Strukturen eines Git-Projekts, welches aufgrund der Anpassung der pom.xml durch jQAssistant gescannt und in einer Neo4j-Instanz gespeichert wurde
* Daten zu CVE's über Dateiimport und Request an die cveapi (letzteres nur über einen übersichtlichen Datensatz möglich, da die Firewall des API-Servers den Request zum Schutz vor Denial-of-Service-Attacken sperrt)

### konkrete Quellen
* Projektgrundlage: Petclinic --> verwendete Frameworks & zugehörige Versionen
* CVE-Dateiimport: historische CVE's von 2021 als JSON-Datei (Stand: 23.08.2021) --> Download älterer Dateien unter https://nvd.nist.gov/vuln/data-feeds möglich
* cve-api: Import der 20 aktuellsten CVE's

### mögliche Schwachstellen
* Artefakte/Frameworks ohne Versionsnummer, Versionsnummer mit unbrauchbaren Präfix/Suffix ("Verunreinheiten")
* Artefakte/Frameworks, welche nicht über jQAssistant erfasst werden konnten
* genaues Matching zwischen verwendete Version und schwachstellenrelevante Version


### Common Vulnerabilities and Exposures (CVE)
* Industriestandard zur Benennung öffentlich bekannter Sicherheitslücken
* gewissen Druck auf die Hersteller der betroffenen Produkte aufgrund öffentlicher Schwachstellen
* in großen Teilen ein Community-Projekt --> MITRE Organization als Head of

## Annahmen
* Die relevanten Artefakte konnten von jQAssistant gescannt und als Artefakte mit den entsprechenden Labels "name" und "version" gespeichert werden. 
* Relevante Daten wie betroffene Konfigurationen, Versionen und Schweregrad der Schwachstellen können über die JSON-Strukturierung der CVE's abgerufen werden.
* Die Artefakte können mit den Informationen der CVE's abgeglichen werden. 

## Validierung

### Datenaufbereitung
* Tabellenansicht bzgl. aller relevanten (im Projekt möglichen) Schwachstellen
* Graphische Übersicht über betroffene Artefakte und Anzahl der zugehörigen CVE's
* Graphische Übersicht über Impact Score der auftretenden CVE's
* Graphische Übersicht über Exploitability Score der auftretenden CVE's

### Aktionen
* Review bzgl. der auftretenden Schwachstellen und dessen Schweregrade
* Validierung & Evaluierung der Schwachstellen durch Domainexperten
* Planung der nächsten Schritte/Meilensteine durch Projekt

## Implementierung
* Identifikation der relevanten Artefakte über Node Label "Artifact" --> Extraktion des Namens & der Version
* Weitere mögliche Artefakte können über die Property-Nodes des Nodes der pom.xml mit der Relationship [:HAS_PROPERTY] identifiziert werden. --> Extraction von Name & Version auch hier möglich
* Merging beider DataFrames um eine vollständige Liste zu erhalten & ggf. Versionsangabe von störenden Anhängen befreien um Versionsvergleich zu vereinfachen
* Extraktion der CVE aus der JSON-Datei --> Auflösung von Verschachtelungen und Kürzung der Spaltenanzahl
* Abgleich der identifizierten Artefakte aus dem Projekt mit den betroffenden Konfiguration aus den CVE
* Resultierendes DataFrame kann als Excel gespeichert & für Visualisierungen genutzt werden.



In [1]:
#Import of all used libraries
import py2neo
import pandas as pd
import numpy as np

import json
from pandas.io.json import json_normalize
#import openpyxl

import urllib3
from urllib3 import request
import json
import certifi

from IPython.display import display, HTML
import pygal

In [2]:
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [3]:
#Verbindung zu neo4j und Speicherung des Graphen in der Variable 'Graph'

graph = py2neo.Graph(host='localhost', user='neo4j', password='neo4j')

In [4]:
#Query um alle Artefakte bzgl. Frameworks zu erhalten
#Bereininung von Duplikaten, Testfiles und unnötigen Präfixes 

query = """
MATCH (artifact:Artifact) WHERE NOT artifact.name contains 'petclinic' AND NOT artifact.type = 'test-jar'
WITH DISTINCT artifact 
Return artifact.name as Artefakt, artifact.version as Version
"""
df_usedArtifacts = pd.DataFrame(graph.run(query), columns=['Artefakt', 'Version'])
df_usedArtifacts['Version'] = df_usedArtifacts['Version'].str.replace('[.-]?[a-zA-Z]+[-]?\w+([.-]?\d*)*$','',regex=True)

In [5]:
#Query zu allen Frameworks, die als Property an der pom.xml gespeichert sind
#Joining beider Dataframes, Duplikaterntfernung

query2 = """
Match (p:Pom)-[:HAS_PROPERTY]->(pr:Property) 
Return pr.name as Artefakt, pr.value as Version
"""
df_PomProperties = pd.DataFrame(graph.run(query2), columns=['Artefakt', 'Version'])
df_PomProperties['Artefakt'] = df_PomProperties['Artefakt'].str.replace('.?[vV]ersion','',regex=True)

df_usedItems = pd.merge(left=df_PomProperties, right=df_usedArtifacts, how='outer', left_on='Artefakt', right_on='Artefakt')
for i in df_usedItems.index:
    version_y = df_usedItems['Version_y'][i]
    version_x = df_usedItems['Version_x'][i]
    if pd.isnull(version_x) and (version_y is not None):
        df_usedItems.loc[i, 'Version_x'] = version_y

df_usedItems = df_usedItems.sort_values(by=['Artefakt','Version_x'], ascending=False, na_position='last')
df_usedItems = df_usedItems.drop_duplicates(subset='Artefakt', keep="first")
df_usedItems = df_usedItems.reset_index()
df_usedItems = df_usedItems.drop(['index'], axis=1)

df_usedItems = df_usedItems.drop(['Version_y'], axis=1)
df_usedItems.columns =['Artefakt', 'Version']
df_usedItems

,Artefakt,Version
0,xmlunit2,2.6.2
1,xmlunit-matchers,2.6.2
2,xmlunit-legacy,2.6.2
3,xmlunit-core,2.6.2
4,xml-path,3.1.1
...,...,...
982,activemq,5.15.7
983,accessors-smart,1.2
984,LatencyUtils,2.0.3
985,HikariCP,3.2.0


## Datenimport über cveapi oder entsprechender JSON
Beide Möglichkeiten besitzen eine ähnliche Datenstrukturierung. Der Request über cveapi erweitert die Strukturierung nur um wenige weitere Key-Value-Paare, weshalb das DataFrame zusätzlicher Anpassung benötigt.

In [6]:
# Laden der statischen CVE-Daten über JSON-Datei
# Download über https://nvd.nist.gov/vuln/data-feeds

api = 'false'

with open('CVE/nvdcve-1.1-2021.json', encoding='utf-8') as staticData:
    jsonData = json.load(staticData)
df_raw = pd.json_normalize(jsonData, record_path =['CVE_Items'])


In [ ]:
# Datenimport über cveapi 
# Import aller CVE's wird nicht empfohlen, da dies von der Firewall 
# zur Prävention von Denial-of-service-Attacken verhindert wird

api = 'true'

http = urllib3.PoolManager(
    cert_reqs="CERT_REQUIRED",
    ca_certs=certifi.where()
)

#Request einer spezifischen CVE
#url ='https://services.nvd.nist.gov/rest/json/cves/1.0?CVE-2020-24616'

#Request der 20 aktuellsten CVE
url ='https://services.nvd.nist.gov/rest/json/cves/1.0?startIndex=20' 

#Request der CVE ab einem definierten Startzeitpunkt
#url ='https://services.nvd.nist.gov/rest/json/cves/1.0?modStartDate=2021-01-0101T00:00:00:000 UTC-05:00


r = http.request('GET', url)
r.status

# JSON-Daten werden ausgewertet & in ein Dictionary gespeichert
jsonData = json.loads(r.data.decode('utf-8'))
df_nested_list = pd.json_normalize(jsonData)
df_raw = df_nested_list.loc[:,df_nested_list.columns.isin(['result.CVE_Items'])]
json_struct = json.loads(df_raw.to_json(orient="records")) 
df_raw = pd.json_normalize(json_struct,record_path =['result.CVE_Items'])

## Datenaufbereitung über DataFrames

In [7]:
#Aufbereitung der Daten zu einer Tabelle mit ID, Beschreibung & Schweregrad der Sicherheitslücke

#DataFrame wird auf 6 Spalten gekürzt & Spalten werden umbenannt (Lesbarkeit)
vulnerableList= df_raw.columns.isin(['cve.CVE_data_meta.ID', 'impact.baseMetricV3.cvssV3.confidentialityImpact', 'impact.baseMetricV3.cvssV3.integrityImpact', 'impact.baseMetricV3.cvssV3.availabilityImpact', 'impact.baseMetricV3.exploitabilityScore', 'impact.baseMetricV3.impactScore'])
df_basic = df_raw.loc[:,vulnerableList]
df_basic.columns =['CVE-ID', 'Confidentially Impact', 'Integrity Impact', 'Availability Impact', 'Exploitability Score', 'Impact Score']


#Neues DF mit den CVE-Beschreibungen, da "cve.description.description_data" ein Dictionary enthält
df_raw2 = df_raw.loc[:,df_raw.columns.isin(['cve.CVE_data_meta.ID', 'cve.description.description_data'])]

#Reload & Manipulation des DataFrames um an die entsprechende Beschreibung zu gelangen
json_struct = json.loads(df_raw2.to_json(orient="records")) 
df_desc = pd.json_normalize(json_struct,record_path =['cve.description.description_data'], meta=['cve.CVE_data_meta.ID'])
df_desc = df_desc.loc[:,df_desc.columns.isin(['value', 'cve.CVE_data_meta.ID'])]
df_desc.columns =['CVE description', 'CVE-ID']

#DF-Join von df_basic & df_desc
basicList = pd.merge(left=df_basic, right=df_desc, left_on='CVE-ID', right_on='CVE-ID')
basicList = basicList[['CVE-ID', 'CVE description', 'Confidentially Impact', 'Integrity Impact', 'Availability Impact', 'Impact Score', 'Exploitability Score']]

In [8]:
#Neues DF mit den Konfigurationsbeschreibungen, da "configurations.nodes" ein Dictionary enthält
newList= df_raw.columns.isin(['cve.CVE_data_meta.ID', 'configurations.nodes'])
df_raw3 = df_raw.loc[:,newList]

#Reload & Manipulation des DataFrames um an die entsprechende fehlerhafte Konfiguration zu gelangen
json_struct = json.loads(df_raw3.to_json(orient="records")) 
df_conf = pd.json_normalize(json_struct,record_path =['configurations.nodes'], meta=['cve.CVE_data_meta.ID'])
json_struct = json.loads(df_conf.to_json(orient="records")) 
df_conf2 = pd.json_normalize(json_struct,record_path =['cpe_match'], meta=['operator', 'cve.CVE_data_meta.ID'])

#Kürzung & Umbenennung der Spalten
if api == 'false':
    df_conf2 = df_conf2.loc[:,df_conf2.columns.isin(['vulnerable', 'cpe23Uri', 'versionEndIncluding', 'versionEndExcluding', 'versionStartIncluding', 'versionStartExcluding', 'operator', 'cve.CVE_data_meta.ID'])]
    df_conf2.columns =['Vulnerable for system?', 'cpe23URI', 'Last version (excl)', 'First version (incl)', 'Last version (incl)', 'First version (excl)', 'Connector/Relation', 'CVE-ID']
    df_conf2 = df_conf2[['CVE-ID', 'Vulnerable for system?', 'Connector/Relation', 'cpe23URI','First version (excl)', 'First version (incl)', 'Last version (excl)', 'Last version (incl)']]
elif api == 'true':
    df_conf2 = df_conf2.rename(columns={"cve.CVE_data_meta.ID": "CVE-ID", "vulnerable": "Vulnerable for system", "cpe23Uri": "cpe23URI", "operator": "Connector/Relation"})
    df_conf2 = df_conf2.drop(columns=['cpe_name'])
 

In [9]:
#DataFrame bzgl. Konfigurationen mit Schwachstellen mit den verwendeten Artefakten scannen
list1 = []
list2 = []
df_result = df_conf2[0:0]
df_result.insert(len(df_result.columns), "verwendetes Artefakt", [])
df_result.insert(len(df_result.columns), "verwendete Version", [])

for j in df_usedItems.index:
    version = df_usedItems['Version'][j]
    artefakt = df_usedItems['Artefakt'][j]
    
    df_search = df_conf2.loc[df_conf2['cpe23URI'].str.contains(':'+artefakt + ':', case=False)]
    if df_search is not None:
        lengthDF = df_search.shape[0]
        for i in range(lengthDF):
            list1.append(artefakt)
            list2.append(version)
        df_search.insert(len(df_search.columns), "verwendetes Artefakt", list1)
        df_search.insert(len(df_search.columns), "verwendete Version", list2)
        list1.clear()
        list2.clear()
    df_result = df_result.append(df_search, ignore_index=True)

### Auflistung der Schwachstellen
Die folgende Tabelle stellt alle möglichen Schwachstellen bzgl. der verwendeten Artefakte in dem gescannten Projekt dar. Es werden neben der CVE-ID & dem Artefakt + Versionsnummer auch eine Beschreibung und die jeweiligen Scores zur Einschätzung der Relevanz und des Schweregrades aufgelistet.
Die Liste wird als Excel abgespeichert.

In [10]:
#Neues DataFrame mit kompakten Daten (ohne Versionenvergleich)
df_compact = df_result[0:0]
df_compact = df_result.drop_duplicates(subset='CVE-ID', keep="first")
df_compact = df_compact.loc[:,df_compact.columns.isin(['CVE-ID', 'verwendetes Artefakt', 'verwendete Version'])]
df_compact = pd.merge(left=basicList, right=df_compact, left_on='CVE-ID', right_on='CVE-ID')
#df_compact.to_excel("result_analysis.xlsx")
df_compact

,CVE-ID,CVE description,Confidentially Impact,Integrity Impact,Availability Impact,Impact Score,Exploitability Score,verwendetes Artefakt,verwendete Version
0,CVE-2021-1998,Vulnerability in the MySQL Server product of O...,NONE,LOW,LOW,2.5,1.2,mysql,8.0.13
1,CVE-2021-2001,Vulnerability in the MySQL Server product of O...,NONE,NONE,HIGH,3.6,1.2,mysql,8.0.13
2,CVE-2021-2002,Vulnerability in the MySQL Server product of O...,NONE,NONE,HIGH,3.6,1.2,mysql,8.0.13
3,CVE-2021-2006,Vulnerability in the MySQL Client product of O...,NONE,NONE,HIGH,3.6,1.6,mysql,8.0.13
4,CVE-2021-2007,Vulnerability in the MySQL Client product of O...,LOW,NONE,NONE,1.4,2.2,mysql,8.0.13
...,...,...,...,...,...,...,...,...,...
125,CVE-2021-3393,An information leak was discovered in postgres...,LOW,NONE,NONE,1.4,2.8,postgresql,42.2.5
126,CVE-2021-34428,"For Eclipse Jetty versions <= 9.4.40, <= 10.0....",LOW,LOW,NONE,2.5,0.9,jetty,9.4.12.v20180830
127,CVE-2021-34429,"For Eclipse Jetty versions 9.4.37-9.4.42, 10.0...",LOW,NONE,NONE,1.4,3.9,jetty,9.4.12.v20180830
128,CVE-2021-34629,The SendGrid WordPress plugin is vulnerable to...,LOW,NONE,NONE,1.4,2.8,sendgrid,4.3.0


### Visualisierung
Im folgenden Abschnitt wird mithilfe einiger Charts der Zusammenhang zwischen CVE, betroffenes Artefakt, Auftrittshäufigkeit und Schweregrade dargestellt.

In [11]:
df_count = df_compact['verwendetes Artefakt'].value_counts().to_frame()
df_count['Artefakt'] = df_count.index
df_count.columns =['Count CVE', 'Artefakt']
df_count.columns.name = None
df_count = df_count.reset_index()
df_count = df_count.loc[:,df_count.columns.isin(['Count CVE', 'Artefakt'])]
df_count

,Count CVE,Artefakt
0,84,mysql
1,9,elasticsearch
2,8,mongodb
3,6,tomcat
4,6,jetty
5,5,postgresql
6,3,netty
7,3,solr
8,1,jackson-databind
9,1,undertow


In [12]:
pie_chart = pygal.Pie()
pie_chart.title = 'Artefakte mit Sicherheitsbedenken'
for i in range(len(df_count)):
    artefakt= df_count['Artefakt'][i]
    anzahl=df_count['Count CVE'][i]
    pie_chart.add(artefakt, anzahl)
display(HTML(base_html.format(rendered_chart=pie_chart.render(is_unicode=True))))

In [13]:
top3 = df_count.head(3)

line_chart = pygal.Bar(show_legend=True, human_readable=True, fill=True, legend_at_bottom=True, print_values=True)

line_chart.title = 'Hohe Risiken bzgl. Top 3 der betroffenen Artefakte'
line_chart.x_labels = ['Confidentially risk', 'Integrity risk', 'Availability risks']

for j in top3.index:
    artefakt = top3['Artefakt'][j]
    df_interimResult = df_compact.loc[df_compact['verwendetes Artefakt'].str.contains(artefakt, case=False)]

    count_ci = len(df_interimResult[df_interimResult['Confidentially Impact'] == 'HIGH'])
    count_ii = len(df_interimResult[df_interimResult['Integrity Impact'] == 'HIGH'])
    count_ai = len(df_interimResult[df_interimResult['Availability Impact'] == 'HIGH'])
    line_chart.add(artefakt, [count_ci, count_ii, count_ai])

    

display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))


In [14]:
df_interimResult = df_compact[0:0]
TopRisk = top3['Artefakt'][0]
labels = ['Confidentially Impact', 'Integrity Impact', 'Availability Impact']
chartvalues = []

radar_chart = pygal.Radar()
radar_chart.title = 'Auswirkungen der Sicherheitslücken bei ' + TopRisk
radar_chart.x_labels = labels
radar_chart.y_labels = [
  {'label': 'High', 'value': 3},
  {'label': 'Medium', 'value': 2},
  {'label': 'Low', 'value': 1},
  {'label': 'None', 'value': 0}]

df_interimResult = df_compact.loc[df_compact['verwendetes Artefakt'].str.contains(TopRisk, case=False)]
df_interimResult = df_interimResult.drop_duplicates(subset='CVE-ID', keep="first")

for i in df_interimResult.index:
    cve_id = df_interimResult['CVE-ID'][i]
    c = 0
    for j in labels:
        if df_interimResult[j][i] == 'HIGH':
            df_interimResult[j][i] = 3
        elif df_interimResult[j][i] == 'MEDIUM':
            df_interimResult[j][i] = 2
        elif df_interimResult[j][i] == 'LOW':
            df_interimResult[j][i] = 1
        elif df_interimResult[j][i] == 'NONE':
            df_interimResult[j][i] = 0

    radar_chart.add(cve_id, [df_interimResult['Confidentially Impact'][i], df_interimResult['Integrity Impact'][i], df_interimResult['Integrity Impact'][i]])
    chartvalues.clear()
    

display(HTML(base_html.format(rendered_chart=radar_chart.render(is_unicode=True))))

c:\python37\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\python37\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
df_interimResult = df_compact[0:0]
cve_list = []

xy_chart = pygal.XY(stroke=False, x_title='Impact Score', y_title='Exploitability Score')
xy_chart.title = 'Score per CVE (Top3)'


for j in top3.index:
    artefakt = top3['Artefakt'][j]
    df_interimResult = df_compact.loc[df_compact['verwendetes Artefakt'].str.contains(artefakt, case=False)]
    
    for i in df_interimResult.index:
        cve_list.append({'value': (df_interimResult['Impact Score'][i], df_interimResult['Exploitability Score'][i]), 'label': df_interimResult['CVE-ID'][i]})
    
    xy_chart.add(artefakt, cve_list)
 

display(HTML(base_html.format(rendered_chart=xy_chart.render(is_unicode=True))))

In [16]:
df_interimResult = df_compact[0:0]
cveListC = []
cveListH = []
cveListM = []
cveListL = []
cveListN = []


treemap_ImpactS = pygal.Treemap()
treemap_ImpactS.title = 'Impact Score per CVE'

for j in df_count.index:
    artefakt = df_count['Artefakt'][j]
    df_interimResult = df_compact.loc[df_compact['verwendetes Artefakt'].str.contains(artefakt, case=False)]
    
    for i in df_interimResult.index:
        impactScore = df_interimResult['Impact Score'][i]
        if impactScore >= 9:
            cveListC.append({'value': df_interimResult['Impact Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore >= 7 and impactScore < 9:
            cveListH.append({'value': df_interimResult['Impact Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore >= 4 and impactScore < 7:
            cveListM.append({'value': df_interimResult['Impact Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore < 4:
            cveListL.append({'value': df_interimResult['Impact Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore == 0:
            cveListN.append({'value': df_interimResult['Impact Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
    
treemap_ImpactS.add('Critical', cveListC)
treemap_ImpactS.add('High', cveListH)
treemap_ImpactS.add('Medium', cveListM)
treemap_ImpactS.add('Low', cveListL)
treemap_ImpactS.add('None', cveListN)
    
 

display(HTML(base_html.format(rendered_chart=treemap_ImpactS.render(is_unicode=True))))

In [17]:
df_interimResult = df_compact[0:0]
cveListC = []
cveListH = []
cveListM = []
cveListL = []
cveListN = []


treemap_ExploitS = pygal.Treemap()
treemap_ExploitS.title = 'Exploitability Score per CVE'

for j in df_count.index:
    artefakt = df_count['Artefakt'][j]
    df_interimResult = df_compact.loc[df_compact['verwendetes Artefakt'].str.contains(artefakt, case=False)]
    
    for i in df_interimResult.index:
        impactScore = df_interimResult['Exploitability Score'][i]
        if impactScore >= 9:
            cveListC.append({'value': df_interimResult['Exploitability Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore >= 7 and impactScore < 9:
            cveListH.append({'value': df_interimResult['Exploitability Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore >= 4 and impactScore < 7:
            cveListM.append({'value': df_interimResult['Exploitability Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore < 4:
            cveListL.append({'value': df_interimResult['Exploitability Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif impactScore == 0:
            cveListN.append({'value': df_interimResult['Exploitability Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
    
treemap_ExploitS.add('Critical', cveListC)
treemap_ExploitS.add('High', cveListH)
treemap_ExploitS.add('Medium', cveListM)
treemap_ExploitS.add('Low', cveListL)
treemap_ExploitS.add('None', cveListN)
    
 

display(HTML(base_html.format(rendered_chart=treemap_ExploitS.render(is_unicode=True))))

## Ergebnisse
### Softwareanalyse
* Die häufigsten CVE bzgl. Petclinic treten aufgrund von der Verwendung von mysql. (2020 unteranderem auch FasterXML jackson-databind)
* Besonders häufig werden Schwachstellen bzgl. der Verfügbarkeit ausgenutzt.
* Einzeln werden zwar die Schweregrad bzgl. Verfügbarkeit, Integrität und Vertraulichkeit vorwiegend als 'High' eingestuft, allerdings wird dies nur bedingt durch den Impact Score und den Exploitability Score vermittelt. --> Wahrnehmung, dass die Risiken noch nicht kritisch "genug" sind.

### Hindernisse/Verbesserungspotentiale
* Die betroffenen Artefaktversionen innerhalb der JSON-Datei/Response werden nicht in zwei Spalten gepflegt. Es wird stattdessen zwischen Startversion inklusive und exklusive und Endversion inklusive und exklusive unterschieden. Dies hat bis dato die weitere Eingrenzung der Liste verhindert, sodass eine manuelle zusätzliche Einkürzung noch notwendig ist.
* Interessant wäre zudem auch eine Relation zwischen betroffene Artefakte, Klassen und Codezeilen. Bei dem Versuch dies über weitere Cypher-Anfragen und DataFrames darzustellen kam es zu keinem zufriedenenstellenden und plausiblen Ergebnis
* Verbesserung der Schnittstelle zur API
* Verbesserung der Regex-Ausdrücke
* Beim Scan durch jQAssistant wurde ersichtlich, dass nicht für jedes Artefakt direkt eine Versionsnummer verfügbar ist. An manchen Stellen werden Variablen wie "${hamcrestVersion}" gepflegt. Je nach Projekt wird zum Beispiel auch eine fehlende Versionsnummer anders gehändelt (z.B. Label bleibt leer oder es wird mit "None" besetzt.) An manchen Stellen wurde dies über das Mergen von Artefakten der pom.xml und Artefakt-Nodes schon gelöst.

## Nächste Schritte
* Präsentation der Ergebnisse und Diskussion mit den Domainexperten und dem Projekt (womöglich einem Teil der Stakeholder)
* Sichtung der Ergebnistabelle und weitere Kürzung der CVE-Liste
* Planung & Aufwandsschätzung von möglichen Updates & Bugfixes
* Regelmäßige CVE-Scans planen

## Quellen

- Software Analytics Canvas von Markus Harrer (https://www.feststelltaste.de/software-analytics-canvas/)
- https://nvd.nist.gov/vuln/data-feeds